# Authentication Narrative

This workbook, will use the Velociraptor Auth artefacts and provide a narration of the events based on the search criteria you provide. 

### Environment Setup

Install the necessary packages

In [ ]:
#%%capture
%pip install --upgrade opensearch-py 

### Import Components

Import the necessary packages

In [ ]:
#Perform the necessary imports.
from opensearchpy import OpenSearch
from opensearchpy.helpers import scan

#Date
import datetime

#Password 
from getpass import getpass

#JSON
import json

### User Variables

The following block of code contains the variables that can be modified by the user. See comments for details

In [ ]:
#Un-comment the right one
#vr_artifact_type = "acsc-auth"
vr_artifact_type = "rdp-auth"

# You can use * as a wildcard. Don't adjust the keys
# Logon Types (Not exhaustive)
# 2 - Local Logon (At the terminal)
# 3 - Network Logon
# 10 - Remote Desktop
search_fields = {
    "UserName": "", #A Username to search on (i.e. this username signed into X machines)
    "SourceIP": "", # A Source to search on (i.e. all users who signed in from X)
    "Computer": "", # A Target (i.e. all users who signed into X machine)
    "LogonType": "", # The type of logon (i.e. show me all RDP logons. Types above)
    "EventID": "" #The event ID type (i.e. only show me 4625 events (failed logons))
}

#Should be in ISO8601 - e.g. 2023-08-12T06:04:32.188Z
date_from = ""
date_to = ""

username = 'admin' #Username, there is always an admin account in DFIR2Go

### System Variables

DO NOT MODIFY BELOW THIS LINE!!! Unless you know what you are doing. 

> When you run this block, a text input field will appear ⬆️. Enter a valid password for the OpenSearch Admin account

In [ ]:
hosts = [{"host": "os01", "port": 9200}] #Opensearch details
password = getpass() #Prompt the user for the password 
auth = (username, password) #Create the authentication details

ca_certs_path = '/certs/ca/ca.crt' #Certs path
logon_types = {
    "2": "Interactive (Console)",
    "3": "Network",
    "4": "Batch (Scheduled Task)",
    "5": "Service",
    "7": "Unlock",
    "8": "Network Clear Text (Basic Auth?)",
    "9": "Alternative Credentials Provided (RunAs?)",
    "10": "Remote Interactive (Remote Desktop)",
    "11": "Cached Interactive (Away from authentication source)"
}

vr_index_name = {
    'acsc-auth':'logstash-vr-acsc-auth*',
    'rdp-auth':'logstash-vr-rdpauth*',
}

### OpenSearch connection

Makes the connection to OpenSearch.

In [ ]:
client = OpenSearch(
    hosts=hosts, 
    http_compress=True,
    http_auth=auth,
    use_ssl=True,
    timeout=300,
    verify_certs=True,
    ssl_assert_hostname=False,
    ssl_show_warn=False,
    ca_certs=ca_certs_path        
)

### Perform the search and display results

The following block will build the search query, perform the search and display the results.
See comments for specific details

In [ ]:
index = vr_index_name[vr_artifact_type] #The index you want to search.

should_clauses = []
for field, term in search_fields.items():
  if term:
    if "*" in term:
      should_clauses.append({ "bool": { "should": [ { "query_string": { "fields": [field], "query": term } } ], "minimum_should_match": 1 }})
    else:
      should_clauses.append({ "bool": { "should": [ { "match": {field: term} } ], "minimum_should_match": 1 } })

# Initialize filter_list 
filter_list = [ 
      { 
        "bool": { 
          "filter": should_clauses 
        } 
      }] 
      
# Conditionally add the range filter if dates are provided 
if date_from and date_to: 
  filter_list.append({ 
    "range": { 
      "@timestamp": { 
        "gte": date_from, 
        "lte": date_to, 
        "format": "strict_date_optional_time" 
      } 
    }})

#The query is the search to perform. The following will return an entire index. 
#To assist in building your query, you can use the developer tools in the browser. 
#When performing a search, review the submitted webpage which will contain a query value which can be used as a starting point.
query = {
  "sort": [
    {
      "@timestamp": {
        "order": "asc",
        "unmapped_type": "boolean"
      }
    }
  ],
    "size": "10000",  
    "timeout": "300s",
    "query": {
        "bool": {
            "must": [],
            "filter": should_clauses,
            "should": [],
            "must_not": []
          }
    }
}

#Use Scan function to interact with OpenSearch, once the results have been returned, loop through all of them printing the _source component.
for results in scan(client, query=query, index=index):
    event_id = results["_source"]["EventID"]

    #Now work through the various event IDs
    if event_id == 21:
      # Terminal Services - Successful Authentication
      pass

    elif event_id == 22:
      # Terminal Services - Shell Start
      pass

    elif event_id == 23:
      # Terminal Services - Log off
      pass

    elif event_id == 24:
      # Terminal Services - Session Disconnect
      pass

    elif event_id == 25:
      # Terminal Services - Reconnect to RDP Session
      pass
      
    elif event_id == 39:
      # Terminal Services - Disconnected by another user
      pass

    elif event_id == 40:
      # Terminal Services - Disconnected
      print("At {time}, the {user} disconnected {computer} via {logon_type} from {source}.".format(
          time=results['_source']['EventTime'], user=results['_source']['UserName'], computer=results['_source']['Computer'], 
          logon_type=logon_types[results['_source']['LogonType']], source=results['_source']['SourceIP']))

    elif event_id == 1194:
      # Remote Desktop - User auth successful
      pass

    elif event_id == 4624:
      # Security - Successful Logon
      print("At {time}, the {user} successfully logged onto {computer} via {logon_type} from {source}.".format(
          time=results['_source']['EventTime'], user=results['_source']['UserName'], computer=results['_source']['Computer'], 
          logon_type=logon_types[results['_source']['LogonType']], source=results['_source']['SourceIP']))

    elif event_id == 4625:
      # Security - Failed Logon
      print("At {time}, the {user} failed to log onto {computer} via {logon_type} from {source}.".format(
          time=results['_source']['EventTime'], user=results['_source']['UserName'], computer=results['_source']['Computer'], 
          logon_type=logon_types[results['_source']['LogonType']], source=results['_source']['SourceIP']))

    elif event_id == 4634 or event_id == 4636 or event_id == 4647:
      # Security - Account was logged off
      # TODO Add duration 
      print("At {time}, the {user} logged off from {computer}".format(
          time=results['_source']['EventTime'], user=results['_source']['UserName'], computer=results['_source']['Computer']))
    

    else:
      # Default action if EventID is not found 
      print("An unexpected Event ID was used. That Event ID was: {eventid}".format(eventid=results['_source']['EventID']))
    